In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pyspark
from pyspark.sql import SparkSession

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy
import matplotlib.pyplot as plt

In [3]:
spark = SparkSession.builder.appName("bank_info").getOrCreate()

23/11/20 20:41:41 WARN Utils: Your hostname, admins-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.4.50 instead (on interface en0)
23/11/20 20:41:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/20 20:41:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/20 20:41:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
path = "Loan_default.csv"

23/11/20 20:41:54 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [10]:
bank_df = spark.read.option("header",'True').option('delimiter', ',').csv(path)
bank_df.show()

+----------+---+------+----------+-----------+--------------+--------------+------------+--------+--------+-----------+--------------+-------------+-----------+-------------+-----------+-----------+-------+
|    LoanID|Age|Income|LoanAmount|CreditScore|MonthsEmployed|NumCreditLines|InterestRate|LoanTerm|DTIRatio|  Education|EmploymentType|MaritalStatus|HasMortgage|HasDependents|LoanPurpose|HasCoSigner|Default|
+----------+---+------+----------+-----------+--------------+--------------+------------+--------+--------+-----------+--------------+-------------+-----------+-------------+-----------+-----------+-------+
|I38PQUQS96| 56| 85994|     50587|        520|            80|             4|       15.23|      36|    0.44| Bachelor's|     Full-time|     Divorced|        Yes|          Yes|      Other|        Yes|      0|
|HPSK72WA7R| 69| 50432|    124440|        458|            15|             1|        4.81|      60|    0.68|   Master's|     Full-time|      Married|         No|           N

In [15]:
num_rows = bank_df.count()
num_columns = bank_df.columns
print("Number of rows:", num_rows)
print("Number of columns:", len(num_columns))

Number of rows: 255347
Number of columns: 18


In [11]:
bank_df.printSchema()

root
 |-- LoanID: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Income: string (nullable = true)
 |-- LoanAmount: string (nullable = true)
 |-- CreditScore: string (nullable = true)
 |-- MonthsEmployed: string (nullable = true)
 |-- NumCreditLines: string (nullable = true)
 |-- InterestRate: string (nullable = true)
 |-- LoanTerm: string (nullable = true)
 |-- DTIRatio: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- EmploymentType: string (nullable = true)
 |-- MaritalStatus: string (nullable = true)
 |-- HasMortgage: string (nullable = true)
 |-- HasDependents: string (nullable = true)
 |-- LoanPurpose: string (nullable = true)
 |-- HasCoSigner: string (nullable = true)
 |-- Default: string (nullable = true)



In [23]:
from pyspark.sql.functions import avg


In [26]:
avg_age = bank_df.agg(avg("Age")).first()[0]
print("The average age of this dataset is {:.1f}.".format(avg_age))

The average age of this dataset is 43.5.


In [27]:
bank_df.createOrReplaceTempView("bank_table")

In [28]:
# Calculate average DTIRatio
avg_dti_result = spark.sql("SELECT AVG(DTIRatio) as avg_dti FROM bank_table")
avg_dti = avg_dti_result.first()["avg_dti"]

# Calculate average Default
avg_default_result = spark.sql("SELECT AVG(Default) as avg_default FROM bank_table")
avg_default = avg_default_result.first()["avg_default"]

# Count records with Default=1
count_default_df_result = spark.sql("SELECT COUNT(*) as count_default FROM bank_table WHERE Default = 1")
count_default_df = count_default_df_result.first()["count_default"]

# Count total records
bank_count_result = spark.sql("SELECT COUNT(*) as bank_count FROM bank_table")
bank_count = bank_count_result.first()["bank_count"]

# Display the results
print(f"The total number of records in this dataset is: {bank_count}")
print(f"The average Debt to income ratio across the data set is: {avg_dti}")
print(f"The average default rate is: {avg_default}, meaning of the {bank_count} records {count_default_df} defaulted.")

The total number of records in this dataset is: 255347
The average Debt to income ratio across the data set is: 0.5002120643673129
The average default rate is: 0.11612824901017048, meaning of the 255347 records 29653 defaulted.


In [7]:
bank_df.describe()

DataFrame[summary: string, LoanID: string, Age: string, Income: string, LoanAmount: string, CreditScore: string, MonthsEmployed: string, NumCreditLines: string, InterestRate: string, LoanTerm: string, DTIRatio: string, Education: string, EmploymentType: string, MaritalStatus: string, HasMortgage: string, HasDependents: string, LoanPurpose: string, HasCoSigner: string, Default: string]

In [8]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [ ]:
# Converting PySpark DataFrame to Pandas DataFrame
# import pandas as pd
# pandas_df = dataframe.toPandas() 